Initialize app with `firebase_config.json`, `firebase_private.json` file

In [33]:
import firebase_admin
from firebase_admin import credentials, initialize_app, auth, exceptions

if not firebase_admin._apps:
    credential = credentials.Certificate('../../firebase_private.json')
    initialize_app(credential)

In [34]:
import pyrebase, json

pb = pyrebase.initialize_app(json.load(open('../../firebase_config.json')))

### Create a user
By default, Firebase Authentication will generate a random uid for the new user. If you instead want to specify your own uid for the new user, you can include it as an argument passed to the user creation method, it must be a string between 1-128 characters long.

In [35]:
user_data = {
    'email': 'f163d45b667c4f46@example.com',
    'password': 'f163d45b667c4f46'
}
try:
    new_user = auth.create_user(**user_data)
except exceptions.FirebaseError as exc:
    if exc.code == 'ALREADY_EXISTS':
        user = auth.get_user_by_email(user_data['email'])
        new_user = auth.update_user(uid=user.uid, **user_data)

auth.set_custom_user_claims(new_user.uid, {'role':'MANAGER', 'lab': None})

new_user.__dict__

{'_data': {'localId': 'Hjwx99OtZwdKDq0yJie6Ys0nKpg2',
  'email': 'f163d45b667c4f46@example.com',
  'passwordHash': 'UkVEQUNURUQ=',
  'emailVerified': False,
  'passwordUpdatedAt': 1681879375110,
  'providerUserInfo': [{'providerId': 'password',
    'federatedId': 'f163d45b667c4f46@example.com',
    'email': 'f163d45b667c4f46@example.com',
    'rawId': 'f163d45b667c4f46@example.com'}],
  'validSince': '1681879375',
  'disabled': False,
  'createdAt': '1681879375110'}}

### Verify ID tokens

In [36]:
# Sign in with email and password to retrive ID tokens
res = pb.auth().sign_in_with_email_and_password(**user_data)

decoded_token = auth.verify_id_token(res['idToken'])
print(decoded_token)

{'role': 'MANAGER', 'lab': None, 'iss': 'https://securetoken.google.com/mrianalysisstudioformouse', 'aud': 'mrianalysisstudioformouse', 'auth_time': 1681879376, 'user_id': 'Hjwx99OtZwdKDq0yJie6Ys0nKpg2', 'sub': 'Hjwx99OtZwdKDq0yJie6Ys0nKpg2', 'iat': 1681879376, 'exp': 1681882976, 'email': 'f163d45b667c4f46@example.com', 'email_verified': False, 'firebase': {'identities': {'email': ['f163d45b667c4f46@example.com']}, 'sign_in_provider': 'password'}, 'uid': 'Hjwx99OtZwdKDq0yJie6Ys0nKpg2'}


### Retrieve user data
The primary way to identify a user is by their uid. The Firebase Admin SDK also supports looking up user information with phone number and email.

In [37]:
user = auth.get_user(new_user.uid)
user = auth.get_user_by_email(new_user.email)

print('Display name:', user.display_name)
print('Email:', user.email)
print('User ID:', user.uid)
print('Role:', user.custom_claims.get('role'))
print('Lab:', user.custom_claims.get('lab'))

Display name: None
Email: f163d45b667c4f46@example.com
User ID: Hjwx99OtZwdKDq0yJie6Ys0nKpg2
Role: MANAGER
Lab: None


### List all users

In [38]:

page = auth.list_users()
for user in auth.list_users(max_results=5).users:
    print('User: ' + user.uid)

User: 890zFbqtC3OkJuFvUaMqbNM0Q2I3
User: BfnOj9buPQaxx1ElsZpFe5jE6bV2
User: HAnY63BeqyQqjvun4BuYcjdl80A3
User: Hjwx99OtZwdKDq0yJie6Ys0nKpg2
User: OLqrg5fD4HYPoWpO7j04ePAQqda2


### Update a user
You need to specify a uid along with the properties to update for that user

In [39]:
new_user_data = {'display_name': 'Name update'}
user = auth.update_user(new_user.uid, **new_user_data)
user.__dict__

{'_data': {'localId': 'Hjwx99OtZwdKDq0yJie6Ys0nKpg2',
  'email': 'f163d45b667c4f46@example.com',
  'displayName': 'Name update',
  'passwordHash': 'UkVEQUNURUQ=',
  'emailVerified': False,
  'passwordUpdatedAt': 1681879375110,
  'providerUserInfo': [{'providerId': 'password',
    'displayName': 'Name update',
    'federatedId': 'f163d45b667c4f46@example.com',
    'email': 'f163d45b667c4f46@example.com',
    'rawId': 'f163d45b667c4f46@example.com'}],
  'validSince': '1681879375',
  'disabled': False,
  'lastLoginAt': '1681879376519',
  'createdAt': '1681879375110',
  'customAttributes': '{"role": "MANAGER", "lab": null}',
  'lastRefreshAt': '2023-04-19T04:42:56.519Z'}}

### Set and validate custom user claims
Custom claims can contain sensitive data, therefore they should only be set from a privileged server environment by the Firebase Admin SDK.

In [40]:
user_claims = {'role': 'ADMIN', 'lab': 'Lab test'}
auth.set_custom_user_claims(new_user.uid, user_claims)


user = auth.get_user(new_user.uid)
print('Role:', user.custom_claims.get('role'))
print('Lab:', user.custom_claims.get('lab'))
# Check user role
if user.custom_claims.get("role") == 'ADMIN':
    print(f'User {new_user.uid} can access to admin resource')

Role: ADMIN
Lab: Lab test
User Hjwx99OtZwdKDq0yJie6Ys0nKpg2 can access to admin resource


### Delete a user

In [41]:
auth.delete_user(new_user.uid)
print('Successfully deleted user')

Successfully deleted user
